# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

## Importaciones

In [ ]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

In [3]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## 1. Solución aleatoria

In [ ]:
class FirefighterAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.carrying_victim = False
        self.knocked_down = False

        # self.positions_not_available = [] ????

    
    def step(self):
        print("Teamo")


    def check_walls_doors_PAST(self):
        """Verifica si existen paredes o puertas en las celdas adyacentes."""
        x, y = self.pos

        # Obtener cuatro posible celdas
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        for puerta in self.model.doors:
            # Verificar si coincide la posición actual con una de las dos de la puerta
            if puerta[0] == (x, y) or puerta[1] == (x, y):
                # Estado de la puerta
                if puerta[2] == True:
                    print("Pasa algo")
                else:
                    # Si está cerrada, no puede pasar
                    print("Pipipipi")


    """ ¿Cuál es el objetivo de esta función?
        Opción por si se quiere para moverse """
    def check_walls_doors(self):
        """ Verifica si existen paredes o puertas en las celdas adyacentes. """
        x, y = self.pos

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Verificación de paredes --> 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        cell = self.model.walls[x][y]
        index = 0
        indexes_walls = []
        for i in cell:
            # Sí hay una pared
            if i == 1 or i == 0.5:
                indexes_walls.append(index)
            index += 1

        # Verificación de puertas
        # --> Puede haber acceso a dos puertas desde una misma celda ??
        cell = self.model.doors[x][y]
        index = 0
        indexes_doors = []
        for i in cell:
            # Sí hay una puerta y está cerrada
            if i == 2:
                indexes_doors.append(index)
            index += 1

        # Qué sigue? Quitarlas de possible_positions ?


    """Supuesto de que no puede encontrarse dos puertas en una misma celda
        A menos de que se la pase como argumento también la celda a la que se quiere mover"""
    
    def toggle_door(self, x, y):
        """ Cambia el estado de la puerta en la celda (x, y). """
        cell = self.model.doors[x][y]

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Encontrar la puerta que se quiere alterar
        # 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        index = 0
        for i in cell:
            if i != 0:
                if i == 1:
                    cell[index] = 2
                    new_state = 2
                else:
                    cell[index] = 1
                    new_state = 1
                break
            index += 1
        
        # Encontrar la otra celda de la puerta
        if index == 0:
            other_cell = possible_positions[0]
            other_index = 2
        elif index == 2:
            other_cell = possible_positions[2]
            other_index = 0
        elif index == 1:
            other_cell = possible_positions[3]
            other_index = 1
        else:
            other_cell = possible_positions[1]
            other_index = 3
        
        # Cambiar el estado de la otra celda
        cell = self.model.doors[other_cell[0]][other_cell[1]]
        cell[other_index] = new_state




            


    